In [1]:
import pandas as pd
import numpy as np
from process_and_ml.where_is import WhereIs

In [2]:
where_is = WhereIs(False)

In [3]:
all_paths = where_is.get_paths_list

In [4]:
%%time
customers = pd.read_csv(all_paths[1], sep=';')

CPU times: user 4.51 s, sys: 483 ms, total: 4.99 s
Wall time: 5.29 s


<string>:2: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.


In [5]:
###Load With Rapids nvidia rtx2060 for notebooks



In [6]:
customers.shape

(191652, 369)

In [7]:
customers.head()

,LNR,AGER_TYP,AKT_DAT_KL,ALTER_HH,ALTER_KIND1,ALTER_KIND2,ALTER_KIND3,ALTER_KIND4,ALTERSKATEGORIE_FEIN,ANZ_HAUSHALTE_AKTIV,...,VK_ZG11,W_KEIT_KIND_HH,WOHNDAUER_2008,WOHNLAGE,ZABEOTYP,PRODUCT_GROUP,CUSTOMER_GROUP,ONLINE_PURCHASE,ANREDE_KZ,ALTERSKATEGORIE_GROB
0,9626,2,1.0,10.0,NaN,NaN,NaN,NaN,10.0,1.0,...,2.0,6.0,9.0,7.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,1,4
1,9628,-1,9.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,0.0,9.0,NaN,3,FOOD,SINGLE_BUYER,0,1,4
2,143872,-1,1.0,6.0,NaN,NaN,NaN,NaN,0.0,1.0,...,11.0,6.0,9.0,2.0,3,COSMETIC_AND_FOOD,MULTI_BUYER,0,2,4
3,143873,1,1.0,8.0,NaN,NaN,NaN,NaN,8.0,0.0,...,2.0,NaN,9.0,7.0,1,COSMETIC,MULTI_BUYER,0,1,4
4,143874,-1,1.0,20.0,NaN,NaN,NaN,NaN,14.0,7.0,...,4.0,2.0,9.0,3.0,1,FOOD,MULTI_BUYER,0,1,3


In [8]:
#checking different values
diff_values = customers['ALTER_KIND1'].value_counts()

In [9]:
#Counting values
diff_values.values.sum()

11766

In [10]:
#counting NaNs
customers['ALTER_KIND1'].isna().sum()

179886

In [11]:
columns_name = list(customers.columns)

In [12]:
test_column_name = columns_name[1]
test_column_name

'AGER_TYP'

In [13]:
test_column_name.split('_',1)[0]

'AGER'

## We can see we have much more NaNs than values on columns, so it's hard to filled the Nans, so we need to find another strategy.

In [58]:
#let's try to separate columns by similar names
diff_before_underline = {col.split('_',1)[0] for col in columns_name}


In [15]:
diff_before_underline

{'AGER',
 'AKT',
 'ALTER',
 'ALTERSKATEGORIE',
 'ANREDE',
 'ANZ',
 'ARBEIT',
 'BALLRAUM',
 'CAMEO',
 'CJT',
 'CUSTOMER',
 'D19',
 'DSL',
 'EINGEFUEGT',
 'EINGEZOGENAM',
 'EWDICHTE',
 'EXTSEL992',
 'FINANZ',
 'FINANZTYP',
 'FIRMENDICHTE',
 'GEBAEUDETYP',
 'GEBURTSJAHR',
 'GEMEINDETYP',
 'GFK',
 'GREEN',
 'HEALTH',
 'HH',
 'INNENSTADT',
 'KBA05',
 'KBA13',
 'KK',
 'KKK',
 'KOMBIALTER',
 'KONSUMNAEHE',
 'KONSUMZELLE',
 'LNR',
 'LP',
 'MIN',
 'MOBI',
 'NATIONALITAET',
 'ONLINE',
 'ORTSGR',
 'OST',
 'PLZ8',
 'PRAEGENDE',
 'PRODUCT',
 'REGIOTYP',
 'RELAT',
 'RETOURTYP',
 'RT',
 'SEMIO',
 'SHOPPER',
 'SOHO',
 'STRUKTURTYP',
 'TITEL',
 'UMFELD',
 'UNGLEICHENN',
 'VERDICHTUNGSRAUM',
 'VERS',
 'VHA',
 'VHN',
 'VK',
 'W',
 'WOHNDAUER',
 'WOHNLAGE',
 'ZABEOTYP'}

In [16]:
#check how many columns start with the same name
len(diff_before_underline)

66

In [59]:
### Hum, from 367 to 66, so let's create groups.
dict_prefix_cols = {key:[col for col in columns_name if col.split('_',1)[0] == key ] for key in diff_before_underline}

### Very Good now we ll now how many cols with the same prefix we have, let's check the result.


In [15]:
dict_prefix_cols

NameError: name 'dict_prefix_cols' is not defined

### Let's separate the unique columns from multiple prefix columns

In [16]:
dict_multi = dict(filter(lambda elem: len(elem[1])>=2, dict_prefix_cols.items()))

NameError: name 'dict_prefix_cols' is not defined

In [17]:
### Check the result

In [1]:
###Run the info_data_frame
df_column_info = pd.read_excel(all_paths[5], engine='openpyxl')

NameError: name 'pd' is not defined

In [1]:
df_column_attibutes = pd.read_excel(all_paths[4])

NameError: name 'pd' is not defined

In [22]:
### Great we have some columns to study and check relation with common prefix, so ll can see what to do with NaNs
### Let's check how many columns of this groups have only 1 value cross of dataframe

cjt_df = customers[dict_multi['CJT']]

In [23]:
plz_df = customers[dict_multi['PLZ8']]

In [24]:
plz_df.head()

,PLZ8_ANTG1,PLZ8_ANTG2,PLZ8_ANTG3,PLZ8_ANTG4,PLZ8_BAUMAX,PLZ8_GBZ,PLZ8_HHZ
0,3.0,3.0,1.0,0.0,1.0,5.0,5.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,3.0,3.0,1.0,3.0,2.0,3.0
3,3.0,2.0,1.0,0.0,1.0,4.0,3.0
4,2.0,4.0,2.0,1.0,2.0,3.0,3.0


In [25]:
type(plz_df['PLZ8_ANTG1'][0])


numpy.float64

In [26]:
# Separate all keys with more than 2 columns names
%%time
equal_columns = []
for row in customers.itertuples():
    for key,value in dict_multi.items():
        teste_row = row
        if len(set([row.__getattribute__(column_name) for column_name in value])) < 2 :
            equal_columns.append(key)

CPU times: user 41.8 s, sys: 7.27 ms, total: 41.8 s
Wall time: 41.8 s


In [27]:
#Contando as colunas unicas
from collections import Counter

number_unique_values_columns = Counter(equal_columns)

In [52]:
customers.shape

(191652, 369)

In [24]:
#Organizando os Atributes arruamndo o index e o nome das colunas
column_array = df_column_attibutes.iloc[0].values
column_array[0] = 'to_drop'
df_column_attibutes.columns = column_array

In [28]:
#Drop a coluna que veio errada da importação do Excell
df_column_attibutes.drop('to_drop', axis=1, inplace=True)


KeyError: "['to_drop'] not found in axis"

In [36]:
#Excluindo a primeira linha para não ficar repetido o nome da coluna
df_column_attibutes = df_column_attibutes.iloc[1:]

### Criando um dataframe com os valores desconhecidos ou transacoes desconhecidas, para depois transofrmar em NaN

In [56]:
columns_data_to_nan = df_column_attibutes.loc[(df_column_attibutes['Meaning'] == 'unknown') | (df_column_attibutes['Meaning'] == 'no transactions known'), ['Attribute', 'Value']]

In [46]:
##Verificando quantos NaNs temos a columns to fix para ver oque precisamos arrumar.
n_nan_to_fix =columns_data_to_nan.isna().sum()

In [51]:
# Criando um dicionario com os dados faltantes que estão em Nan
dict_last_nan_atribute = {162:'D19_BANKEN_DATUM',
                          196:'D19_BANKEN_OFFLINE_DATUM',
                          206:'D19_BANKEN_ONLINE_DATUM',
                          329:'D19_GESAMT_DATUM',
                          339:'D19_GESAMT_OFFLINE_DATUM',
                          349:'D19_GESAMT_ONLINE_DATUM',
                          502:'D19_GESAMT_ONLINE_DATUM',
                          520:'D19_TELKO_OFFLINE_DATUM',
                          530:'D19_TELKO_ONLINE_DATUM',
                          570:'D19_VERSAND_DATUM',
                          580:'D19_VERSAND_OFFLINE_DATUM',
                          590:'D19_VERSAND_ONLINE_DATUM',
                          }

In [53]:
columns_data_to_nan.loc[columns_data_to_nan.index == 6]

,Attribute,Value
6,ALTERSKATEGORIE_GROB,"-1, 0"


In [57]:
for key, value in dict_last_nan_atribute.items():
    columns_data_to_nan.loc[columns_data_to_nan.index == key, ['Attribute']] = value

In [65]:
columns_data_to_nan.to_csv('./data/wrangler/azdias_costumer_to_nan.csv')

In [67]:
teste=pd.read_csv('./data/wrangler/azdias_costumer_to_nan.csv')

## Next Step setar os Nan e analisar rows columns with NaN